# Introduction

- Nama : Vittorio Damma Putra

- Batch : HCK 031

**Problem**

We are working 


# Import Libraries

In [1]:
# Data handling and manipulation
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# metric evaluation
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Data loading

In [2]:
df = pd.read_csv('multiTimeline.csv')

In [3]:
print("first 10 data:")
display(df.head(10))

first 10 data:


,Category: All categories
Week,Pulau Pari: (Indonesia)
2022-03-27,16
2022-04-03,13
2022-04-10,18
2022-04-17,29
2022-04-24,35
2022-05-01,95
2022-05-08,62
2022-05-15,47
2022-05-22,35


In [4]:
print("last 10 data:")
display(df.tail(10))

last 10 data:


,Category: All categories
2024-06-16,40
2024-06-23,45
2024-06-30,54
2024-07-07,33
2024-07-14,40
2024-07-21,43
2024-07-28,34
2024-08-04,32
2024-08-11,36
2024-08-18,29


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, Week to 2024-08-18
Data columns (total 1 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Category: All categories  127 non-null    object
dtypes: object(1)
memory usage: 2.0+ KB


In [6]:
# checking duplicate columns
df.duplicated().sum()  

81

In [7]:
df.describe()

,Category: All categories
count,127
unique,46
top,28
freq,8


In [8]:
df.index

Index(['Week', '2022-03-27', '2022-04-03', '2022-04-10', '2022-04-17',
       '2022-04-24', '2022-05-01', '2022-05-08', '2022-05-15', '2022-05-22',
       ...
       '2024-06-16', '2024-06-23', '2024-06-30', '2024-07-07', '2024-07-14',
       '2024-07-21', '2024-07-28', '2024-08-04', '2024-08-11', '2024-08-18'],
      dtype='object', length=127)

In [11]:
# select data
df = df[['Category: All categories']]
df.head()

,Category: All categories
Week,Pulau Pari: (Indonesia)
2022-03-27,16
2022-04-03,13
2022-04-10,18
2022-04-17,29


# Data Cleaning

In [16]:
# check date types
df['Category: All categories'].dtypes

# Convert into date type
df['Date'] = pd.to_datetime(df['Category: All categories'], format='%d-%m-%Y')

# sort based on the date
df = df.sort_values('Date')
df.set_index('Date', inplace=True)

# range date, 5 working days index
range_index = pd.date_range(start=df.index.min(), end=df.index.max(),
                            freq='B')
# re-index
df = df.reindex(range_index)

ValueError: time data "Pulau Pari: (Indonesia)" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.